<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/Document_Management/examples_build_knowledge_graph_with_instructor_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ollama langchain_community  --quiet

modelid="llama3.1"
emb_modelid="mxbai-embed-large"
host="localhost:11434"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")
get_ipython().system_raw(f"ollama pull {emb_modelid}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 1.8 MB/s eta 0:00:00


In [ ]:
!pip install instructor --quiet

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional
from graphviz import Digraph
from IPython.display import display

class Node(BaseModel):
    id: int
    label: str
    color: str

class Edge(BaseModel):
    source: int
    target: int
    label: str
    color: str = "black"


class KnowledgeGraph(BaseModel):
    nodes: list[Node] = Field(..., default_factory=list)  # A list of nodes in the knowledge graph.
    edges: list[Edge] = Field(..., default_factory=list)  # A list of edges in the knowledge graph.


    def visualize_knowledge_graph(self):
        dot = Digraph(comment="Knowledge Graph")

        for node in self.nodes:
            dot.node(name=str(node.id), label=node.label, color=node.color)
        for edge in self.edges:
            dot.edge(str(edge.source), str(edge.target), label=edge.label, color=edge.color)

        return display(dot)

In [ ]:
def generate_graph(input) -> KnowledgeGraph:
    return client.chat.completions.create(
        model=modelid,
        messages=[
            {
                "role": "user",
                "content": f"Help me understand the following by describing it as small knowledge graph: {input}",
            }
        ],
        response_model=KnowledgeGraph,
    )

generate_graph("Explain quantum mechanics").visualize_knowledge_graph()
